<a href="https://colab.research.google.com/github/sidfeiner/EpitopeWorkshop/blob/master/EpitopeWorkshop_Omer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
token = getpass('insert token')
!git clone https://$token@github.com/sidfeiner/EpitopeWorkshop.git
!mv EpitopeWorkshop workshop

insert token··········
Cloning into 'EpitopeWorkshop'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 227 (delta 117), reused 162 (delta 59), pack-reused 0
Receiving objects: 100% (227/227), 10.61 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [2]:
!cd workshop/ && pip3 install -e .
import sys
sys.path.append('/content/workshop/')
!pip install fsspec

Obtaining file:///content/workshop
     |████████████████████████████████| 2.3 MB 15.5 MB/s 
     |████████████████████████████████| 135 kB 73.0 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 41 kB 192 kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=3484f6f5201b4d3a6c30534ece8a27506a801c85615a26d209894380f8f4051f
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50466 sha256=dce9aa42dbe924bab8d54100021efa865a0232801ea951838c9398c760640c27
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995dbf11db6cede602e7ef
  Created wheel for quantiprot: filename=quantiprot-0.2.4-py3-none-any.whl size=151137 sha256=2fd7f935879167220ae6949126ddfc41a6a6489c124046a317e0c99caf4ca809
  Stored in directory: /root/.cache/pip/wheels/93/8d/

In [3]:
from EpitopeWorkshop.process import read
from EpitopeWorkshop.common.conf import DEFAULT_WINDOW_SIZE, DEFAULT_WITH_SLIDING_WINDOW
from EpitopeWorkshop.process.features import FeatureCalculator
import dask.dataframe as dd

sequences_file_path = '/content/workshop/data/iedb_linear_epitopes.fasta'
window_size = 9
partitions_amt=2

df = read.load_fasta_fasta(sequences_file_path, True, window_size, 1)
ddf = dd.from_pandas(df, npartitions=partitions_amt)
calculator = FeatureCalculator()
df = calculator.calculate_features(ddf)

In [4]:
from EpitopeWorkshop.common import utils
import torch
import torchtext
from EpitopeWorkshop.common import contract
from EpitopeWorkshop.dataset.EpitopeDataset import EpitopeDataset

sub_df = df[contract.NETWORK_LABELED_INPUT_ARGS]
labels_df = df[contract.IS_IN_EPITOPE_COL_NAME]
ds = EpitopeDataset(sub_df)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ds_train, ds_valid, ds_test = ds.splits()
print(len(ds_train), len(ds_valid), len(ds_test))

3753 524 1015


In [5]:
from torchtext import data
BATCH_SIZE = 25
dl_train, dl_valid, dl_test = ds.iters(batch_size=32)

x0, y0 = ds[0]